# 🗺️ 네이버 플레이스 크롤링 v4.3 (구글 코랩)

## 🎯 v4.3 주요 개선사항
- ✅ **ngrok URL 생성 간소화** (더 빠르고 안정적)
- ✅ **타임아웃 방지** (URL이 바로 표시됩니다)
- ✅ **주소 수집 강화** (15개 셀렉터)
- ✅ **CSV 다운로드 기능**
- ✅ **타지역업체 자동 감지**

## 📖 사용 방법
1. 아래 셀들을 **순서대로** 실행
2. 마지막 셀에서 **URL이 즉시 표시됩니다!** ⚡
3. URL 클릭하여 접속

## 1️⃣ 패키지 설치

In [ ]:
print("🚀 v4.3 설치 시작...\n")
!pip install -q flask playwright pyngrok nest-asyncio
!playwright install chromium
!playwright install-deps chromium
print("\n✅ 설치 완료!")

## 2️⃣ ngrok 토큰 (선택사항)

In [ ]:
NGROK_TOKEN = ""  # 선택사항
if NGROK_TOKEN:
    !ngrok authtoken {NGROK_TOKEN}

## 3️⃣ 크롤러 모듈

In [ ]:
%%writefile naver_crawler.py
from playwright.sync_api import sync_playwright
import time
import re
from urllib.parse import quote

class NaverPlaceCrawlerV43:
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None
    
    def start(self):
        self.playwright = sync_playwright().start()
        self.browser = self.playwright.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-setuid-sandbox', '--disable-dev-shm-usage']
        )
        self.page = self.browser.new_page()
        self.page.set_viewport_size({"width": 1920, "height": 1080})
        return True
    
    def search_places(self, keyword, max_results=20):
        if not self.page:
            self.start()
        
        try:
            url = f"https://map.naver.com/p/search/{quote(keyword)}"
            self.page.goto(url, timeout=20000, wait_until="domcontentloaded")
            time.sleep(3)
            
            iframe = self.page.frame(name="searchIframe")
            if not iframe:
                return []
            
            time.sleep(2)
            for i in range(5):
                iframe.evaluate("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(0.8)
            
            items = []
            for selector in ['li[role="listitem"]', 'li.UEzoS', 'ul._2py9K li']:
                items = iframe.query_selector_all(selector)
                if items:
                    break
            
            if not items:
                return []
            
            results = []
            for idx, item in enumerate(items[:max_results]):
                try:
                    name = self._get_text(item, ['.TYaxT', '.place_bluelink', '.YwYLL'])
                    if not name:
                        continue
                    
                    addr = self._get_text(item, [
                        '.LDgIH', '.addr', 'span.place_addr', '.Osdwn',
                        '[class*="addr"]', 'span[class*="address"]'
                    ])
                    
                    if not addr:
                        html = item.inner_html()
                        patterns = [
                            r'([가-힣]+(?:특별시|광역시|시|도)\s+[가-힣]+(?:구|군|시)\s+[가-힣0-9\s\-]+)',
                            r'(서울[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(경기[^<>]+?(?:동|로|가|길)\s*\d*)',
                        ]
                        for pattern in patterns:
                            match = re.search(pattern, html)
                            if match:
                                addr = match.group(1).strip()
                                break
                    
                    phone = self._get_text(item, ['.dry6Z', '.tel', '[class*="tel"]'])
                    rating = self._get_text(item, ['.h69bs', '.score'])
                    reviews = self._get_text(item, ['.Tvqnp', '.cnt'])
                    reviews = re.sub(r'[^0-9]', '', reviews) if reviews else "0"
                    category = self._get_text(item, ['.YzBgS', '.category'])
                    
                    is_other = self._is_other_region(name, addr, phone, rating, keyword)
                    
                    results.append({
                        'name': name,
                        'category': category if category else "미분류",
                        'address': addr if addr else "주소 정보 없음",
                        'phone': phone if phone else "전화번호 없음",
                        'rating': rating if rating else "",
                        'reviews': reviews,
                        'is_other_region': is_other,
                        'place_type': '타지역업체' if is_other else '주업체'
                    })
                except:
                    continue
            
            return results
        except:
            return []
    
    def _get_text(self, parent, selectors):
        for sel in selectors:
            try:
                el = parent.query_selector(sel)
                if el:
                    text = el.inner_text().strip()
                    if text:
                        return text
            except:
                pass
        return ""
    
    def _is_other_region(self, name, addr, phone, rating, keyword):
        score = 0
        if phone and '070' in phone:
            score += 3
        if addr and addr != "주소 정보 없음":
            if len(addr.split()) <= 3 or addr.endswith(('동', '구', '시')):
                score += 2
        if not addr or addr == "주소 정보 없음":
            score += 2
        if not rating:
            score += 1
        if name and keyword:
            words = [w for w in keyword.split() if len(w) > 1]
            if any(w in name for w in words):
                score += 2
        return score >= 4
    
    def close(self):
        try:
            if self.browser:
                self.browser.close()
            if self.playwright:
                self.playwright.stop()
        except:
            pass

## 4️⃣ 웹 서버 실행 ⚡

**v4.3 개선:** ngrok URL이 즉시 표시됩니다!

In [ ]:
import nest_asyncio
from flask import Flask, request, jsonify, Response
import threading
import time
import io
import csv
from datetime import datetime
from naver_crawler import NaverPlaceCrawlerV43

nest_asyncio.apply()

app = Flask(__name__)
crawler = None

@app.route('/')
def index():
    return '''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>네이버 플레이스 v4.3</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { 
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }
        .header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }
        h1 { font-size: 2em; }
        .content { padding: 40px; }
        .search-box { display: flex; gap: 10px; margin-bottom: 30px; }
        input { flex: 1; padding: 15px; border: 2px solid #ddd; border-radius: 10px; font-size: 16px; }
        button { padding: 15px 30px; background: #667eea; color: white; border: none; border-radius: 10px; cursor: pointer; }
        button:hover { background: #5568d3; }
        .results { margin-top: 30px; }
        .place-card { border: 2px solid #e0e0e0; padding: 20px; margin: 15px 0; border-radius: 12px; }
        .place-card.other { border-color: #ff9800; background: #fff3e0; }
        .badge { display: inline-block; padding: 5px 12px; border-radius: 15px; font-size: 0.85em; margin-left: 10px; }
        .badge.main { background: #4caf50; color: white; }
        .badge.other { background: #ff9800; color: white; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🗺️ 네이버 플레이스 크롤링 v4.3</h1>
            <p>주소 수집 강화 | CSV 다운로드 | 타지역업체 자동 감지</p>
        </div>
        <div class="content">
            <div class="search-box">
                <input type="text" id="keyword" placeholder="검색어 입력 (예: 강남역 맛집)">
                <button onclick="search()">🔍 검색</button>
                <button onclick="downloadCSV()" id="btnDownload" style="display:none; background:#11998e">📥 CSV</button>
            </div>
            <div id="results" class="results"></div>
        </div>
    </div>
    <script>
        let currentResults = [];
        let currentKeyword = '';
        
        async function search() {
            const keyword = document.getElementById('keyword').value.trim();
            if (!keyword) { alert('검색어를 입력하세요'); return; }
            
            document.getElementById('results').innerHTML = '⏳ 검색 중...';
            document.getElementById('btnDownload').style.display = 'none';
            
            try {
                const response = await fetch('/api/search', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ keyword, max_results: 20 })
                });
                const data = await response.json();
                
                if (data.success) {
                    currentResults = data.results;
                    currentKeyword = keyword;
                    
                    const html = data.results.map((p, i) => {
                        const cls = p.is_other_region ? 'place-card other' : 'place-card';
                        const badge = p.is_other_region ? '<span class="badge other">🟠 타지역</span>' : '<span class="badge main">🟢 주업체</span>';
                        return `
                            <div class="${cls}">
                                <strong>[${i+1}] ${p.name}</strong>${badge}<br><br>
                                📍 ${p.address}<br>
                                📞 ${p.phone}<br>
                                ${p.rating ? '⭐ ' + p.rating + ' (' + p.reviews + '개)' : ''}
                            </div>
                        `;
                    }).join('');
                    
                    document.getElementById('results').innerHTML = html;
                    document.getElementById('btnDownload').style.display = 'inline-block';
                } else {
                    document.getElementById('results').innerHTML = '❌ ' + data.error;
                }
            } catch (error) {
                document.getElementById('results').innerHTML = '❌ ' + error.message;
            }
        }
        
        async function downloadCSV() {
            const response = await fetch('/api/download-csv', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ results: currentResults, keyword: currentKeyword })
            });
            const blob = await response.blob();
            const url = window.URL.createObjectURL(blob);
            const a = document.createElement('a');
            a.href = url;
            a.download = `naver_${currentKeyword}_${Date.now()}.csv`;
            a.click();
        }
        
        document.getElementById('keyword').addEventListener('keypress', (e) => {
            if (e.key === 'Enter') search();
        });
    </script>
</body>
</html>'''

@app.route('/api/search', methods=['POST'])
def api_search():
    global crawler
    data = request.json
    keyword = data.get('keyword', '')
    
    if not crawler:
        crawler = NaverPlaceCrawlerV43()
        crawler.start()
    
    results = crawler.search_places(keyword, data.get('max_results', 20))
    return jsonify({'success': True, 'results': results})

@app.route('/api/download-csv', methods=['POST'])
def download_csv():
    data = request.json
    output = io.StringIO()
    writer = csv.DictWriter(output, fieldnames=['name', 'category', 'address', 'phone', 'rating', 'reviews', 'place_type'])
    writer.writeheader()
    writer.writerows(data.get('results', []))
    return Response(output.getvalue().encode('utf-8-sig'), mimetype='text/csv')

# Flask 시작
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

time.sleep(2)

# ngrok 간소화 (v4.3)
print("🌐 ngrok 터널 생성 중...\n")

try:
    from pyngrok import ngrok
    
    # 기존 터널 종료
    ngrok.kill()
    time.sleep(1)
    
    # 새 터널 생성 (간단한 방법)
    public_url = ngrok.connect(5000, bind_tls=True)
    
    # URL 추출 (간소화)
    if isinstance(public_url, str):
        url = public_url
    else:
        url = str(public_url)
    
    print("="*70)
    print("✅ v4.3 서버 시작 완료!")
    print("="*70)
    print()
    print(f"🌐 접속 URL: {url}")
    print()
    print("💡 위 URL을 클릭하세요!")
    print()
    print("🧪 테스트 검색어: 강남역 맛집, 홍대 카페, 스타벅스")
    print()
    print("="*70)
    
except Exception as e:
    print(f"❌ ngrok 오류: {e}")
    print("\n대안: Colab의 포트 포워딩 기능 사용")
    print("또는 ngrok 토큰을 설정하세요: https://ngrok.com")

# 서버 유지
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    if crawler:
        crawler.close()
    print("\n✅ 종료")

## 🎉 완료!

### v4.3 개선사항
- ✅ ngrok URL 생성 간소화 (즉시 표시)
- ✅ 타임아웃 방지
- ✅ `ngrok.kill()` 추가 (기존 터널 정리)
- ✅ `bind_tls=True` 명시

**이제 URL이 즉시 나옵니다!** ⚡